# IA25_P01_G06 — Project 01: Class Timetable (CSP)

#Introduction
Creating academic schedules is a complex challenge for higher education institutions. At IPCA, this manual process consumes significant resources and is prone to errors. This project aims to develop an automated solution based on Artificial Intelligence techniques to generate optimized schedules that satisfy all academic constraints.

**Objectives:**

The objectives are to formulate the schedule problem for an institution, implement hard and soft constraints, and generate a schedule for the dataset provided using constraint satisfaction (CSP)

**Teammates:**

- Ana Paula Paula Canuto da Silva - a24178
- Daniel Filipe Alves Vilmin - a28003
- Eva Alexandra Pereira Gomes - a27484
- Glória Ribeiro Chaves Martins - a22719
- Thiago Yabuki de Araujo - a24207

Link para GitHub: https://github.com/yabukithiago/IA_25_P01_G06


#Agent Design


The scheduling problem is implemented as a Constraint Satisfaction Problem (CSP) using the python-constraint library.
Each scheduling decision—assigning a class (turma) and course (curso) to a specific time slot—is treated as a variable.

##Variables:
Each variable represents a tuple (class, course, session_index), created in the function criar_variaveis().
**If a course has only one weekly class, it is assigned a single session; otherwise, two sessions are created per week**.

##Domains:
The domains, defined in criar_dominios(), consist of all possible time slots (1–20), which correspond to 5 days × 4 time blocks per day (**each block corresponds to two hours**).
For each variable, **unavailable time slots are removed based on teacher constraints (tr)**, ensuring that classes are only scheduled when the corresponding teacher is available.

##Hard Constraints:
Implemented in criar_problema(), these must always be satisfied:


*  Each class group cannot have two classes at the same time
* A teacher cannot teach two courses simultaneously
* Each class group can have a maximum of 3 sessions per day.
* Online classes follow special rules:

      If there are ≤ 3 online sessions, all must occur on the same day.
      If there are > 3 online sessions, at most 3 can occur on the same day.

##Soft Constraints:
Activated when USAR_SOFT_CONSTRAINTS = True, these improve schedule quality but can be relaxed if necessary:

* Sessions of the same course should occur on different days.
* Each class should preferably have classes on no more than four days per week.
* Classes should preferably be consecutive within a day to reduce idle gaps.
* The number of classrooms used by each class should be minimised.

## Heuristics:
They are implemented as soft constraints, designed to guide the CSP solver toward high-quality timetable.

In [58]:


# ============================================================
# SISTEMA DE GESTÃO DE HORÁRIOS - GOOGLE COLAB
# ============================================================
# Sistema de resolução de problemas de constraint satisfaction (CSP)
# para criação automática de horários escolares.
#
# Autores: Eva Gomes, Ana Silva, Thiago Yabuki, Daniel Vilmin, Glória Martins
# Data: 02-11-2025
# ============================================================

#  INSTALAR BIBLIOTECA
!pip install python-constraint -q

# ============================================================
# PARSER - Leitura de Dados
# ============================================================
# Esta secção lê e processa o ficheiro de texto com os dados do problema
# ============================================================


def ler_dados_texto(texto):
    """
    Lê dados a partir de um txt e organiza-os num dicionário.
    Cada secção do texto (marcada por tags como #cc, #dsd, etc.) é processada e
    guardada na estrutura correspondente.
    """

    # Dicionário principal onde serão guardados todos os dados lidos.
    # Cada chave representa uma categoria de informação diferente.
    dados = {
        'cc': {},   # Cursos por turma (ex.: T1 -> [MAT, FIS])
        'olw': [],  # Cursos com apenas uma aula por semana (one lesson per week)
        'dsd': {},  # Cursos atribuídos a cada professor (ex.: P1 -> [MAT, FIS])
        'tr': {},   # Restrições de horário (tempos indisponíveis) de cada professor
        'rr': {},   # Cursos e respetivas salas fixas (ex.: LAB1 para INF)
        'oc': {},   # Cursos e índices das aulas online
        'rooms': [] # Lista geral de salas disponíveis
    }

    # Divide o texto completo em linhas individuais e remove espaços desnecessários
    linhas = texto.strip().split('\n')
    secao_atual = None  # Variável que identifica a secção atual de leitura (ex.: 'cc', 'dsd', etc.)


    for linha in linhas:
        linha = linha.strip()

        # Ignora linhas vazias, cabeçalhos ou separadores
        if not linha or linha.startswith('#head') or linha.startswith('—'):
            continue

        # Cada secção no ficheiro começa por um marcador
        # Este bloco identifica qual secção está a ser lida neste momento.
        if linha.startswith('#'):
            secao_map = {
                '#cc': 'cc',
                '#olw': 'olw',
                '#dsd': 'dsd',
                '#tr': 'tr',
                '#rr': 'rr',
                '#oc': 'oc',
                '#rooms': 'rooms'
            }
            # Verifica qual marcador corresponde à linha atual
            for key, val in secao_map.items():
                if linha.startswith(key):
                    secao_atual = val
                    break
            continue


        # Divide a linha em partes separadas por espaços.
        # Exemplo: "T1 MAT FIS" -> ["T1", "MAT", "FIS"]
        partes = linha.split()
        if not partes:
            continue

        # --- Secção #cc: cursos por turma ---
        # Exemplo de linha: "T1 MAT FIS"
        if secao_atual == 'cc' and len(partes) >= 2:
            # O primeiro elemento é o nome da turma, os restantes são os cursos dessa turma
            dados['cc'][partes[0]] = partes[1:]

        # --- Secção #olw: cursos com uma aula por semana ---
        # Exemplo de linha: "MAT FIS"
        elif secao_atual == 'olw':
            dados['olw'].extend(partes)

        # --- Secção #dsd: cursos por professor ---
        # Exemplo de linha: "P1 MAT FIS"
        elif secao_atual == 'dsd' and len(partes) >= 2:
            # O primeiro elemento é o professor, os restantes são os cursos atribuídos a esse professor
            dados['dsd'][partes[0]] = partes[1:]

        # --- Secção #tr: restrições de horário por professor ---
        # Exemplo de linha: "P1 3 7 8" (professor P1 não está disponível nos blocos 3, 7 e 8)
        elif secao_atual == 'tr' and len(partes) >= 2:
            # Converte os índices de tempo (números) em inteiros
            dados['tr'][partes[0]] = list(map(int, partes[1:]))

        # --- Secção #rr: restrições de sala ---
        # Exemplo de linha: "MAT LAB1" (a disciplina MAT deve ocorrer na sala LAB1)
        elif secao_atual == 'rr' and len(partes) == 2:
            dados['rr'][partes[0]] = partes[1]

        # --- Secção #oc: aulas online ---
        # Exemplo de linha: "MAT 2 5" (aulas 2 e 5 de MAT são online)
        elif secao_atual == 'oc' and len(partes) >= 2:
            curso = partes[0]
            indices = list(map(int, partes[1:]))
            # Se o curso já tiver uma aula online, adiciona os novos índices de aulas online
            if curso in dados['oc']:
                dados['oc'][curso].extend(indices)
            else:
                dados['oc'][curso] = indices

        # --- Secção #rooms: lista de salas disponíveis ---
        # Exemplo de linha: "SalaA SalaB SalaC"
        elif secao_atual == 'rooms':
            dados['rooms'].extend(partes)


    # Se o ficheiro de entrada não tiver uma secção #rooms,
    # define-se uma lista padrão de salas genéricas.
    if not dados['rooms']:
        dados['rooms'] = ['SalaA', 'SalaB', 'SalaC', 'SalaD', 'SalaE']

    # Retorna o dicionário completo com todos os dados estruturados
    return dados


#Imprime o que definimos na funçao anterior
def mostrar_dados(dados):
    """Mostra os dados carregados."""
    print("\n CURSOS POR TURMA:")
    for turma, cursos in dados['cc'].items():
        print(f"  {turma}: {', '.join(cursos)}")

    if dados['olw']:
        print("\n CURSOS COM UMA AULA POR SEMANA:")
        print(f"  {', '.join(dados['olw'])}")

    print("\n CURSOS POR PROFESSOR:")
    for prof, cursos in dados['dsd'].items():
        print(f"  {prof}: {', '.join(cursos)}")

    print("\n RESTRIÇÕES DE HORÁRIO:")
    for prof, slots in dados['tr'].items():
        print(f"  {prof}: blocos indisponíveis {', '.join(map(str, slots))}")

    if dados['rr']:
        print("\n RESTRIÇÕES DE SALA:")
        for curso, sala in dados['rr'].items():
            print(f"  {curso}: {sala}")

    if dados['oc']:
        print("\nAULAS ONLINE:")
        for curso, indices in dados['oc'].items():
            print(f"  {curso}: aula(s) {', '.join(map(str, indices))}")

    print(f"\n SALAS DISPONÍVEIS: {', '.join(dados['rooms'])}")


# ============================================================
# MODEL - Criação e Resolução do Problema
# ============================================================
# Nesta secção, define-se o modelo  do problema de horários.
# Ele inclui:
#   - As variáveis de decisão- elementos sobre os quais vamos tomar decisões (ex: aulas, cursos, turmas).
#   -Os domínios - os valores possíveis que cada variável pode assumir
#   -As restrições (constraints)- regras que todas as soluções devem respeitar
# ============================================================

from constraint import Problem, AllDifferentConstraint
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
import numpy as np

# CONFIGURAÇÃO BASE DO HORÁRIO
DIAS_SEMANA = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
HORARIOS = ["09h-11h", "11h-13h", "14h-16h", "16h-18h"]  #HARD CONSTRAINT- Cada aula dura 2h
BLOCOS_POR_DIA = 4
TOTAL_DIAS = 5

# FUNÇÃO: criar_variaveis(dados)
# ------------------------------------------------------------
# Esta função define as variáveis de decisão do problema CSP.

def criar_variaveis(dados):
    """Cria as variáveis: (turma, curso, aula_idx)."""
    variables = []

    # Percorre todas as turmas e os respetivos cursos
    for turma, cursos in dados['cc'].items():
        for curso in cursos:
            # Verifica se o curso tem apenas uma aula por semana (definido em 'olw')
            n_aulas = 1 if curso in dados.get('olw', []) else 2

            # Cria uma variável por aula
            for i in range(1, n_aulas + 1):
                variables.append((turma, curso, i))

    # Mostrar no terminal todas as variáveis criadas
    print("\n VARIÁVEIS CRIADAS:")
    for var in variables:
        turma, curso, idx = var
        print(f"  {var}")
    print(f"Total: {len(variables)} variáveis\n")

    return variables


# FUNÇÃO: criar_dominios(variables, dados)
# ------------------------------------------------------------
# Esta função define os domínios das variáveis.

def criar_dominios(variables, dados):
    domains = {}

    # Existem 20 slots totais (5 dias × 4 blocos por dia)
    all_slots = list(range(1, 21))

    # Para cada variável (turma, curso, aula)
    for var in variables:
        turma, curso, aula_idx = var

        # Identifica qual o professor responsável por este curso
        prof = None
        for p, cursos_prof in dados.get('dsd', {}).items():
            if curso in cursos_prof:
                prof = p
                break

        # Obtém os blocos onde o professor está indisponível
        indisponiveis = dados.get('tr', {}).get(prof, [])

        # O domínio da variável será todos os blocos disponíveis,
        # exceto aqueles onde o professor não pode lecionar.
        domain = [s for s in all_slots if s not in indisponiveis]

        # Associa o domínio à variável
        domains[var] = domain

    return domains


def criar_problema(dados):
    """Cria e configura o problema CSP."""
    # ============================================================
    # Esta função constrói o **modelo completo** do problema CSP.
    # Aqui são reunidos:
    #   • As variáveis do problema (aulas, turmas, cursos)
    #   • Os domínios de cada variável (horários possíveis)
    #   • As restrições (constraints) que definem o que é uma solução válida
    #
    # O objeto `Problem()` da biblioteca `python-constraint`
    # é o motor que vai procurar combinações de horários que respeitem
    # todas as restrições definidas.
    # ============================================================

    problem = Problem()  # Criação do objeto principal de resolução de restrições

    # Criar variáveis e respetivos domínios a partir dos dados fornecidos
    variables = criar_variaveis(dados)
    domains = criar_dominios(variables, dados)

    # Se não houver variáveis, termina a função
    if not variables:
        return None

    # Adicionar cada variável ao problema com o seu domínio permitido
    for var in variables:
        problem.addVariable(var, domains[var])


    # ****** HARD CONSTRAINTS *********
    # As Hard Constraints são regras obrigatórias.

    # 1️⃣ Aulas de uma turma em slots diferentes
    # Garante que uma turma não tem duas aulas marcadas no mesmo horário.
    # (Todas as variáveis da turma têm de ter valores diferentes.)
    for turma in dados['cc'].keys():
        turma_vars = [v for v in variables if v[0] == turma]
        if len(turma_vars) > 1:
            problem.addConstraint(AllDifferentConstraint(), turma_vars)

    # 2️⃣ Professor não dá duas aulas ao mesmo tempo
    for prof, cursos_prof in dados.get('dsd', {}).items():
        prof_vars = [v for v in variables if v[1] in cursos_prof]
        if len(prof_vars) > 1:
            problem.addConstraint(AllDifferentConstraint(), prof_vars)


    # 3️⃣ Máximo 3 aulas por dia (por turma)
    for turma in dados['cc'].keys():
        turma_vars = [v for v in variables if v[0] == turma]

        # Função auxiliar usada como restrição personalizada
        def max_3_por_dia(*slots):
            contagem = [0] * TOTAL_DIAS
            for s in slots:
                # Cada slot corresponde a um bloco (1–20)
                # O dia é determinado pela divisão pelo número de blocos por dia
                dia = (s - 1) // BLOCOS_POR_DIA
                contagem[dia] += 1
                if contagem[dia] > 3:  # Se ultrapassar 3, a restrição é violada
                    return False
            return True

        if turma_vars:
            problem.addConstraint(max_3_por_dia, turma_vars)

    # 4️⃣ Aulas online: distribuição específica por dia
    # Esta regra controla como as aulas online são distribuídas no horário.
    #   -Se uma turma tiver até 3 aulas online - todas no mesmo dia.
    #   -Se tiver mais de 3 - no máximo 3 por dia (espalhadas na semana).
    print(f"  Cursos com aulas online definidas: {list(dados.get('oc', {}).keys())}")
    for curso, indices in dados.get('oc', {}).items():
        print(f"    {curso}: índices {indices}")

    for turma in dados['cc'].keys():
        # Identificar todas as aulas online desta turma
        aulas_online_turma = []

        for var in variables:
            v_turma, v_curso, v_aula_idx = var
            if v_turma == turma:
                # Verificar se esta aula é considerada online
                online_indices = dados.get('oc', {}).get(v_curso, [])
                if not isinstance(online_indices, list):
                    online_indices = [online_indices] if online_indices else []

                if v_aula_idx in online_indices:
                    aulas_online_turma.append(var)

        if aulas_online_turma:
            print(f"  -Turma {turma}: {len(aulas_online_turma)} aulas online no total")
            for var in aulas_online_turma:
                print(f"      - {var[1]} (variável nº={var[2]})")

            # Caso 1: até 3 aulas online - todas no mesmo dia
            if len(aulas_online_turma) <= 3:
                def todas_online_mesmo_dia(*slots):
                    dias = [(s - 1) // BLOCOS_POR_DIA for s in slots]
                    # set(dias) remove duplicados - se só houver um, estão todas no mesmo dia
                    return len(set(dias)) == 1

                problem.addConstraint(todas_online_mesmo_dia, aulas_online_turma)

            # Caso 2: mais de 3 aulas online - máximo 3 por dia
            else:
                def max_3_online_por_dia(*slots):
                    contagem_por_dia = [0] * TOTAL_DIAS
                    for s in slots:
                        dia = (s - 1) // BLOCOS_POR_DIA
                        contagem_por_dia[dia] += 1
                        if contagem_por_dia[dia] > 3:
                            return False
                    return True

                problem.addConstraint(max_3_online_por_dia, aulas_online_turma)



    # ****** SOFT CONSTRAINTS *****
    # As Soft Constraints não são obrigatórias,

    USAR_SOFT_CONSTRAINTS = True  # Permite ativar ou desativar esta secção

    if USAR_SOFT_CONSTRAINTS:

        # 1️⃣ Aulas da mesma disciplina em dias diferentes
        # Objetivo: evitar que uma disciplina tenha várias aulas no mesmo dia.
        cursos_por_turma = defaultdict(list)
        for var in variables:
            turma, curso, _ = var
            cursos_por_turma[(turma, curso)].append(var)

        for aulas in cursos_por_turma.values():
            if len(aulas) > 1:
                def dias_diferentes(*slots):
                    # Calcula os dias correspondentes a cada slot
                    dias = [(s - 1) // BLOCOS_POR_DIA for s in slots]
                    # Retorna True apenas se todos os dias forem diferentes
                    return len(dias) == len(set(dias))
                problem.addConstraint(dias_diferentes, aulas)



        # 2️⃣ Preferir 4 dias de aulas apenas
        for turma in dados['cc'].keys():
            turma_vars = [v for v in variables if v[0] == turma]

            def preferir_4_dias(*slots):
                # Calcula o conjunto de dias usados
                dias = set((s - 1) // BLOCOS_POR_DIA for s in slots)
                # Retorna True se a turma tiver aulas em no máximo 4 dias
                return len(dias) <= 4

            if turma_vars:
                problem.addConstraint(preferir_4_dias, turma_vars)


        # ------------------------------------------------------------
        # 3️⃣ Preferir aulas consecutivas
        # Objetivo: evitar horários com buracos entre aulas.

        for turma in dados['cc'].keys():
            turma_vars = [v for v in variables if v[0] == turma]

            # Só aplica se a turma tiver entre 2 e 4 aulas (por simplicidade)
            if 2 <= len(turma_vars) <= 4:
                def preferir_consecutivas(*slots):
                    # Agrupar as aulas por dia
                    aulas_por_dia = defaultdict(list)
                    for s in slots:
                        dia = (s - 1) // BLOCOS_POR_DIA   # Determina o dia
                        horario = (s - 1) % BLOCOS_POR_DIA  # Determina o bloco dentro do dia
                        aulas_por_dia[dia].append(horario)

                    # Verificar se há pelo menos um par de aulas consecutivas
                    for dia, horarios in aulas_por_dia.items():
                        if len(horarios) >= 2:
                            horarios_sorted = sorted(horarios)
                            tem_consecutivo = any(
                                horarios_sorted[i+1] - horarios_sorted[i] == 1
                                for i in range(len(horarios_sorted)-1)
                            )
                            # Se houver pelo menos um par consecutivo, é aceitável
                            if tem_consecutivo:
                                return True

                    # Se não tiver múltiplas aulas no mesmo dia, também é aceitável
                    return True

                try:
                    problem.addConstraint(preferir_consecutivas, turma_vars)
                except:
                    # Caso ocorra erro (por ex., variáveis insuficientes),
                    # ignora esta constraint sem comprometer o resto do problema.
                    pass

    return problem



#4️⃣ Tenta que uma turma tenha o minimo de salas associadas a ela
def atribuir_salas(solution, dados):
    # Agrupar por slot
    aulas_por_slot = defaultdict(list)
    for (turma, curso, aula_idx), slot in solution.items():
        aulas_por_slot[slot].append((turma, curso, aula_idx))

    # Dicionário de atribuição de salas
    atribuicao_salas = {}
    salas_disponiveis = dados.get('rooms', ['SalaA', 'SalaB', 'SalaC', 'SalaD', 'SalaE'])

    # Mapear salas já atribuídas a cada turma (para reutilizar)
    salas_por_turma = defaultdict(list)

    for slot in sorted(aulas_por_slot.keys()):
        aulas = aulas_por_slot[slot]
        salas_usadas_neste_slot = []

        for turma, curso, aula_idx in aulas:
            # Verificar se curso tem sala específica (#rr)
            if curso in dados.get('rr', {}):
                sala = dados['rr'][curso]
            else:
                # SOFT CONSTRAINT: Tentar reutilizar uma sala já usada por esta turma
                sala_escolhida = None

                # 1. Verificar se já usou alguma sala que está disponível neste slot
                for sala_anterior in salas_por_turma[turma]:
                    if sala_anterior not in salas_usadas_neste_slot:
                        sala_escolhida = sala_anterior
                        break

                # 2. Se não conseguiu reutilizar, escolher uma nova sala
                if sala_escolhida is None:
                    for sala in salas_disponiveis:
                        if sala not in salas_usadas_neste_slot:
                            sala_escolhida = sala
                            # Registar que esta turma agora usa esta sala
                            if sala not in salas_por_turma[turma]:
                                salas_por_turma[turma].append(sala)
                            break

                # 3. Se todas as salas estão ocupadas neste slot, usar ciclo
                if sala_escolhida is None:
                    sala_idx = len(salas_usadas_neste_slot) % len(salas_disponiveis)
                    sala_escolhida = salas_disponiveis[sala_idx]

                sala = sala_escolhida
                salas_usadas_neste_slot.append(sala)

            atribuicao_salas[(turma, curso, aula_idx, slot)] = sala

    return atribuicao_salas


def preencher_quadro(solution, dados):
    """
    Cria um quadro horário completo, preenchido de forma cronológica,

    Parametros:
        solution (dict): Dicionário com o resultado do solver, no formato:
                         {(turma, curso, aula_idx): slot, ...}
        dados (dict): Dados originais do problema (turmas, cursos, salas, etc.)

    Retorna:
        quadro (list): Estrutura bidimensional (tabela) com todos os dias,
                       horários e aulas atribuídas a cada slot.
    """

    # Inicializar o quadro vazio (matriz 4x5 → 4 horários x 5 dias)
    quadro = []
    for i in range(BLOCOS_POR_DIA):
        linha = []
        for j in range(TOTAL_DIAS):
            # Cada célula do quadro contém informações sobre o dia, hora e aulas
            linha.append({
                "dia": DIAS_SEMANA[j],
                "horario": HORARIOS[i],
                "slot": j * BLOCOS_POR_DIA + i + 1,  # Cálculo do número do slot global (1–20)
                "aulas": []  # Lista de aulas que ocorrerão neste slot
            })
        quadro.append(linha)

    # Agrupar aulas por curso e ordenar cronologicamente
    # O solver pode criar aulas com índices (1, 2, ...) que não estão em ordem temporal,
    # por isso é necessário reordená-las conforme os slots (para exibir corretamente no horário).
    # O solver (a biblioteca de restrições) cria uma solução com pares do tipo:
    # (turma, curso, índice_da_aula) → slot atribuído
    #
    # Exemplo:
    # ('T1', 'Matemática', 1): 10  → aula 1 de Matemática ficou no slot 10
    # ('T1', 'Matemática', 2): 2   → aula 2 de Matemática ficou no slot 2
    #
    # Como o solver apenas garante que as restrições são cumpridas (e não a ordem temporal),
    # o índice da aula (1, 2, 3...) pode não estar na sequência cronológica correta.
    # Por isso, vamos reordenar as aulas segundo os "slots" (ordem temporal).

    aulas_por_curso = defaultdict(list)
    for (turma, curso, aula_idx), slot in solution.items():
        aulas_por_curso[(turma, curso)].append({
            'slot': slot,
            'aula_idx_original': aula_idx
        })


    # Neste ponto, aulas_por_curso tem este aspeto:
    # {
    #   ('T1', 'Matemática'): [
    #       {'slot': 10, 'aula_idx_original': 1},
    #       {'slot': 2, 'aula_idx_original': 2}
    #   ],
    #   ('T1', 'Português'): [
    #       {'slot': 5, 'aula_idx_original': 1}
    #   ]
    # }
    #
    # Ou seja: está agrupado por curso, mas não está em ordem cronológica.



    # Criar novo índice cronológico para cada aula (ex: 1ª aula, 2ª aula)
#    (serve para o horário final ficar por ordem cronológica, não pela ordem que o solver deu)

    aulas_reordenadas = {}
    for (turma, curso), aulas in aulas_por_curso.items():
              # Ordenamos as aulas de cada curso conforme o slot (ordem temporal real)
        aulas.sort(key=lambda x: x['slot'])  # Ordenar por ordem temporal
                # Agora enumeramos novamente as aulas (1ª, 2ª, 3ª...) mas pela ordem temporal.
        for idx, aula in enumerate(aulas, start=1):
            # Guardar mapeamento (turma, curso, slot) por ordem cronológica
            aulas_reordenadas[(turma, curso, aula['slot'])] = idx

    #  Atribuir salas às aulas
    # A função `atribuir_salas()` (definida noutro ponto do código)
    # decide que sala pertence a cada aula, considerando restrições (online, etc.)
    atribuicao_salas = atribuir_salas(solution, dados)
    for (turma, curso, aula_idx_original), slot in solution.items():
        # Calcular em que dia e horário o slot se encontra
        dia_index = (slot - 1) // BLOCOS_POR_DIA      # Índice do dia (0–4)
        horario_index = (slot - 1) % BLOCOS_POR_DIA   # Índice do bloco horário

        # Obter a ordem cronológica da aula (1ª, 2ª, etc.)
        aula_idx_cronologico = aulas_reordenadas[(turma, curso, slot)]

        # Verificar se a aula é online
        online_indices = dados.get('oc', {}).get(curso, [])
        if not isinstance(online_indices, list):
            online_indices = [online_indices] if online_indices else []

        # Determinar a sala correta
        # Se o índice da aula estiver marcado como online → "Online"
        # Caso contrário → ir procurar a sala atribuída, ou usar "SalaA" por defeito.
        # Nota: usamos o índice original da variável (não o cronológico)
        #       para corresponder corretamente com a atribuição de salas.
        if aula_idx_original in online_indices:
            sala = "Online"
        else:
            sala = atribuicao_salas.get(
                (turma, curso, aula_idx_original, slot),
                "SalaA"
            )

        # Adicionar esta aula ao quadro na posição correta
        quadro[horario_index][dia_index]["aulas"].append({
            "turma": turma,
            "curso": curso,
            "sala": sala,
            "aula_num": aula_idx_cronologico,  # Ordem cronológica da aula
            "slot": slot
        })

    return quadro


def visualizar_horario(quadro, dados):
    """Mostra o horário formatado no terminal, tanto por turma como o geral."""

    # Para cada turma existente nos dados, será impresso o seu horário individual
    for turma in sorted(dados['cc'].keys()):
        # Cabeçalho visual do horário da turma
        print("\n" + "=" * 110)
        print(f"HORÁRIO DA TURMA {turma}")
        print("=" * 110)

        # Impressão da linha de cabeçalho com os dias da semana
        print(f"{'Horário':<12} |", end="")
        for dia in DIAS_SEMANA:
            print(f" {dia:<18} |", end="")
        print()
        print("-" * 110)

        # Percorrer cada bloco horário (linhas)
        for i in range(BLOCOS_POR_DIA):
            # Mostrar o horário (ex: 9h00–10h00)
            print(f"{HORARIOS[i]:<12} |", end="")
            # Percorrer cada dia (colunas)
            for j in range(TOTAL_DIAS):
                celula = quadro[i][j]
                # Filtrar apenas as aulas da turma atual
                aulas_turma = [a for a in celula["aulas"] if a["turma"] == turma]

                # Se houver aulas nesse horário, mostra o curso e a sala
                if aulas_turma:
                    textos = [f"{a['curso']}({a['sala']})" for a in aulas_turma]
                    conteudo = ", ".join(textos)
                else:
                    # Caso contrário, exibe vazio
                    conteudo = "---"

                # Imprimir o conteúdo formatado dentro da tabela
                print(f" {conteudo:<18} |", end="")
            print()
        print("-" * 110)

    # Após imprimir os horários individuais, cria-se o quadro geral
    print("\n" + "=" * 110)
    print("QUADRO GERAL - TODAS AS TURMAS")
    print("=" * 110)

    # Cabeçalho geral com os dias
    print(f"{'Horário':<12} |", end="")
    for dia in DIAS_SEMANA:
        print(f" {dia:<18} |", end="")
    print()
    print("-" * 110)

    # Impressão do quadro geral com todas as turmas
    for i in range(BLOCOS_POR_DIA):
        print(f"{HORARIOS[i]:<12} |", end="")
        for j in range(TOTAL_DIAS):
            celula = quadro[i][j]

            # Se há aulas nesse slot, mostra as turmas e cursos envolvidos
            if celula["aulas"]:
                textos = [f"{a['turma']}-{a['curso']}" for a in celula["aulas"]]
                # Limita o número de elementos mostrados por célula
                conteudo = ", ".join(textos[:2])
                # Caso existam mais aulas no mesmo slot, adiciona "..."
                if len(celula["aulas"]) > 2:
                    conteudo += "..."
            else:
                conteudo = "---"

            print(f" {conteudo:<18} |", end="")
        print()
    print("-" * 110)



def visualizar_horario_matplotlib(quadro, dados, output_path='horarios_output'):
    """Gera uma visualização gráfica dos horários usando matplotlib."""

    # Importa o módulo necessário para gerar gráficos
    import os
    # Verifica se a pasta de saída existe; se não, cria-a
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print(f"  Pasta criada: {output_path}")


 # Cores para diferentes tipos de aulas
    # Associa cores automaticamente para as salas
    cores_salas = {'Online': '#FF6B6B'}  # Vermelho para Online (fixo)

    # Obter todas as salas únicas (exceto Online)
    salas_unicas = [sala for sala in dados.get('rooms', []) if sala != 'Online']

    # Associa cores distintas usando colormap
    import matplotlib.cm as cm
    if salas_unicas:
      cores_auto = cm.Set3(np.linspace(0, 1, len(salas_unicas)))
      for i, sala in enumerate(salas_unicas):
          # Converter RGBA para HEX
          rgba = cores_auto[i]
          hex_cor = '#{:02x}{:02x}{:02x}'.format(int(rgba[0]*255), int(rgba[1]*255), int(rgba[2]*255))
          cores_salas[sala] = hex_cor

    cores_salas['default'] = '#E0E0E0'  # Cinza para salas não mapeadas

    turmas = sorted(dados['cc'].keys())

    # Criar uma figura para cada turma
    for turma in turmas:
        fig, ax = plt.subplots(figsize=(14, 8))

        # Configurar eixos
        ax.set_xlim(0, len(DIAS_SEMANA))
        ax.set_ylim(0, len(HORARIOS))
        ax.set_xticks(np.arange(len(DIAS_SEMANA)) + 0.5)
        ax.set_yticks(np.arange(len(HORARIOS)) + 0.5)
        ax.set_xticklabels(DIAS_SEMANA, fontsize=12, fontweight='bold')
        ax.set_yticklabels(HORARIOS[::-1], fontsize=10)  # Inverter para 09h ficar no topo
        ax.set_xlabel('')
        ax.set_ylabel('')

        # Grid
        ax.set_xticks(np.arange(len(DIAS_SEMANA) + 1), minor=True)
        ax.set_yticks(np.arange(len(HORARIOS) + 1), minor=True)
        ax.grid(which='minor', color='gray', linestyle='-', linewidth=1.5)
        ax.tick_params(which='minor', size=0)

        # Título
        ax.set_title(f'HORÁRIO DA TURMA {turma}', fontsize=16, fontweight='bold', pad=20)

        # Preencher células
        for i in range(BLOCOS_POR_DIA):
            for j in range(TOTAL_DIAS):
                celula = quadro[i][j]
                aulas_turma = [a for a in celula["aulas"] if a["turma"] == turma]

                # Posição da célula (invertida no eixo y)
                y = len(HORARIOS) - 1 - i
                x = j

                if aulas_turma:
                    # Pegar a cor da sala
                    sala = aulas_turma[0]["sala"]
                    cor = cores_salas.get(sala, cores_salas['default'])

                    # Desenhar retângulo
                    rect = Rectangle((x, y), 1, 1, linewidth=1.5,
                                    edgecolor='gray', facecolor=cor, alpha=0.8)
                    ax.add_patch(rect)

                    # Adicionar texto
                    texto = '\n'.join([f"{a['curso']}" for a in aulas_turma])
                    ax.text(x + 0.5, y + 0.5, texto,
                           ha='center', va='center',
                           fontsize=10, fontweight='bold',
                           wrap=True)
                else:
                    # Célula vazia
                    rect = Rectangle((x, y), 1, 1, linewidth=1.5,
                                    edgecolor='gray', facecolor='white', alpha=0.3)
                    ax.add_patch(rect)

        # Legenda
        legend_elements = []
        salas_usadas = set()
        for horario_linha in quadro:
            for celula in horario_linha:
                for aula in celula["aulas"]:
                    if aula["turma"] == turma:
                        salas_usadas.add(aula["sala"])

        for sala in sorted(salas_usadas):
            cor = cores_salas.get(sala, cores_salas['default'])
            legend_elements.append(mpatches.Patch(facecolor=cor, edgecolor='gray',
                                                  label=sala, alpha=0.8))

        if legend_elements:
            ax.legend(handles=legend_elements, loc='upper left',
                     bbox_to_anchor=(1.02, 1), fontsize=10)

        plt.tight_layout()

        # Guardar figura
        filename = f'{output_path}/horario_{turma}.png'
        plt.savefig(filename, dpi=150, bbox_inches='tight')
        print(f"  Horário de {turma} guardado: {filename}")
        plt.close()

    # Criar quadro geral (todas as turmas)
    fig, ax = plt.subplots(figsize=(16, 10))

    ax.set_xlim(0, len(DIAS_SEMANA))
    ax.set_ylim(0, len(HORARIOS))
    ax.set_xticks(np.arange(len(DIAS_SEMANA)) + 0.5)
    ax.set_yticks(np.arange(len(HORARIOS)) + 0.5)
    ax.set_xticklabels(DIAS_SEMANA, fontsize=12, fontweight='bold')
    ax.set_yticklabels(HORARIOS[::-1], fontsize=10)

    ax.set_xticks(np.arange(len(DIAS_SEMANA) + 1), minor=True)
    ax.set_yticks(np.arange(len(HORARIOS) + 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1.5)
    ax.tick_params(which='minor', size=0)

    ax.set_title('QUADRO GERAL - TODAS AS TURMAS', fontsize=16, fontweight='bold', pad=20)

    # Cores por turma
    cores_turmas = plt.cm.Set3(np.linspace(0, 1, len(turmas)))
    cor_por_turma = {turma: cores_turmas[i] for i, turma in enumerate(turmas)}

    for i in range(BLOCOS_POR_DIA):
        for j in range(TOTAL_DIAS):
            celula = quadro[i][j]
            y = len(HORARIOS) - 1 - i
            x = j

            if celula["aulas"]:
                # Se há múltiplas turmas, dividir a célula
                num_aulas = len(celula["aulas"])
                altura_por_aula = 1.0 / num_aulas

                for idx, aula in enumerate(celula["aulas"]):
                    turma_aula = aula["turma"]
                    cor = cor_por_turma[turma_aula]

                    y_offset = y + (idx * altura_por_aula)
                    rect = Rectangle((x, y_offset), 1, altura_por_aula,
                                    linewidth=0.5, edgecolor='gray',
                                    facecolor=cor, alpha=0.7)
                    ax.add_patch(rect)

                    # Texto
                    texto = f"{turma_aula}-{aula['curso']}"
                    ax.text(x + 0.5, y_offset + altura_por_aula/2, texto,
                           ha='center', va='center', fontsize=7, fontweight='bold')
            else:
                rect = Rectangle((x, y), 1, 1, linewidth=1.5,
                                edgecolor='gray', facecolor='white', alpha=0.3)
                ax.add_patch(rect)

    # Legenda de turmas
    legend_elements = [mpatches.Patch(facecolor=cor_por_turma[turma],
                                     edgecolor='gray', label=turma, alpha=0.7)
                      for turma in turmas]
    ax.legend(handles=legend_elements, loc='upper left',
             bbox_to_anchor=(1.02, 1), fontsize=10, title='Turmas')

    plt.tight_layout()
    filename = f'{output_path}/horario_geral.png'
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    print(f"  Quadro geral guardado: {filename}")
    plt.close()

    print("\n Visualizações gráficas criadas com sucesso!")


    # Tentar exibir as imagens
    try:
        from IPython.display import Image, display
        print("\n" + "=" * 60)
        print("VISUALIZAÇÃO DOS HORÁRIOS")
        print("=" * 60)

        for ficheiro in ficheiros:
            caminho = os.path.join(output_path, ficheiro)
            print(f"\n {ficheiro}:")
            display(Image(filename=caminho))
    except:
        print("\nPara ver as imagens, procure a pasta criada na barra lateral do Colab")


# ============================================================
#  FUNÇÃO PRINCIPAL
# ============================================================

def executar(dataset_texto, nome="Dataset"):
    """Executa o sistema completo de criação e resolução do problema de horários."""
    print("=" * 60)
    print(f"{nome}")
    print("=" * 60)

    # Ler e processar os dados do texto de entrada
    dados = ler_dados_texto(dataset_texto)
    mostrar_dados(dados)

    print("\n")
    print("=" * 60)
    print("A RESOLVER PROBLEMA...")
    print("=" * 60)

    # Criar e configurar o problema CSP
    problem = criar_problema(dados)
    if not problem:
        print("Erro ao criar problema.")
        return

    """ Devido ao tempo demorado a executar- decidimos nao aplicar isto
    # Calcular o número total de soluções possíveis (usando getSolutions)
    print("\nA calcular o número total de soluções possíveis...")
    all_solutions = problem.getSolutions()
    print(f"Número total de soluções encontradas: {len(all_solutions)}")
    """

    # Obter apenas uma solução concreta (usando getSolution)
    print("\nA procurar uma solução válida...")
    solution = problem.getSolution()

    if not solution:
        print("Nenhuma solução encontrada!")
        return

    print("Solução encontrada com sucesso!")

    # Preencher o quadro horário com base na solução obtida
    quadro = preencher_quadro(solution, dados)

    # Mostrar horário no terminal
    visualizar_horario(quadro, dados)

    # Criar visualizações gráficas com matplotlib
    print("\n" + "=" * 60)
    print("A CRIAR VISUALIZAÇÕES GRÁFICAS...")
    print("=" * 60)
    visualizar_horario_matplotlib(quadro, dados)


# ============================================================
# EXECUTAR COM UPLOAD DE FICHEIRO
# ============================================================

from google.colab import files
import io

print("=" * 60)
print("1-Selecione o botão 'Escolher ficheiros'")
print("2-Selecione o seu ficheiro .txt")
print("3️-Aguarde que o upload seja concluído")

uploaded = files.upload()

if uploaded:
    # Apenas faz o upload do primeiro ficheiro enviado
    nome_ficheiro = list(uploaded.keys())[0]
    conteudo = uploaded[nome_ficheiro].decode('utf-8')

    print(f"\n Ficheiro '{nome_ficheiro}' uploaded com sucesso!")
    executar(conteudo, f"Dataset: {nome_ficheiro}")
else:
    print("\n Ficheiro não foi carregado com sucesso")

1-Selecione o botão 'Escolher ficheiros'
2-Selecione o seu ficheiro .txt
3️-Aguarde que o upload seja concluído


Saving dataset.txt to dataset (16).txt

 Ficheiro 'dataset (16).txt' uploaded com sucesso!
Dataset: dataset (16).txt

 CURSOS POR TURMA:
  t01: UC11, UC12, UC13, UC14, UC15
  t02: UC21, UC22, UC23, UC24, UC25
  t03: UC31, UC32, UC33, UC34, UC35

 CURSOS POR PROFESSOR:
  jo: UC11, UC21, UC22, UC31
  mike: UC12, UC23, UC32
  rob: UC13, UC14, UC24, UC33
  sue: UC15, UC25, UC34, UC35

 RESTRIÇÕES DE HORÁRIO:
  mike: blocos indisponíveis 13, 14, 15, 16, 17, 18, 19, 20
  rob: blocos indisponíveis 1, 2, 3, 4
  sue: blocos indisponíveis 9, 10, 11, 12, 17, 18, 19, 20

 RESTRIÇÕES DE SALA:
  UC14: Lab01
  UC22: Lab01

AULAS ONLINE:
  UC21: aula(s) 2
  UC31: aula(s) 2

 SALAS DISPONÍVEIS: SalaA, SalaB, SalaC, SalaD, SalaE


A RESOLVER PROBLEMA...

 VARIÁVEIS CRIADAS:
  ('t01', 'UC11', 1)
  ('t01', 'UC11', 2)
  ('t01', 'UC12', 1)
  ('t01', 'UC12', 2)
  ('t01', 'UC13', 1)
  ('t01', 'UC13', 2)
  ('t01', 'UC14', 1)
  ('t01', 'UC14', 2)
  ('t01', 'UC15', 1)
  ('t01', 'UC15', 2)
  ('t02', 'UC21', 1)
  

# Agent running
Initially, the program included a step to calculate the total number of possible solutions using the getSolutions() method from the python-constraint library. However, this process was found to be extremely time-consuming, especially as the number of variables and constraints increased.

To ensure reasonable execution time and maintain the system’s usability, we decided to use only the getSolution() method instead. This method retrieves a single valid solution that satisfies all the hard and soft constraints, allowing the schedule to be generated efficiently without exhaustively exploring the entire solution space.

This choice balances performance and practicality, providing a feasible and representative solution while avoiding the computational cost of enumerating all possible schedules.

##Analysis of the results
The generated timetables respected all hard constraints.

Soft constraints such as having classes on up to four days per week and sessions of the same course should occur on different days was respected. Preferring consecutive sessions were generally satisfied in the final solutions, although not perfectly optimized.

This shows that the CSP model is correctly defined and that the solver can produce feasible and logical timetables within seconds.

##Future Improvements
Implement a weighted scoring system to better balance soft constraints


#Conclusion
This project successfully created a scheduling system using Constraint Satisfaction Problem (CSP) techniques. The program can automatically generate valid class timetables that respect teacher availability and other rules.

Although we used only one solution to save time, the results show that the system works efficiently and produces logical schedules. In the future, we could improve the agent by adding optimization methods and a user-friendly interface.